In [5]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
job_posting_df = pd.read_csv('/content/drive/MyDrive/hackathon_dataset/jobs.csv')
job_posting_df.head(5)


,Uniq Id,Crawl Timestamp,Job Title,Job Salary,Job Experience Required,Key Skills,Role Category,Location,Functional Area,Industry,Role
0,9be62c49a0b7ebe982a4af1edaa7bc5f,2019-07-05 01:46:07 +0000,Digital Media Planner,Not Disclosed by Recruiter,5 - 10 yrs,Media Planning| Digital Media,Advertising,Mumbai,"Marketing , Advertising , MR , PR , Media Plan...","Advertising, PR, MR, Event Management",Media Planning Executive/Manager
1,3c52d436e39f596b22519da2612f6a56,2019-07-06 08:04:50 +0000,Online Bidding Executive,Not Disclosed by Recruiter,2 - 5 yrs,pre sales| closing| software knowledge| clien...,Retail Sales,"Pune,Pune","Sales , Retail , Business Development","IT-Software, Software Services",Sales Executive/Officer
2,ffad8a2396c60be2bf6d0e2ff47c58d4,2019-08-05 15:50:44 +0000,Trainee Research/ Research Executive- Hi- Tec...,Not Disclosed by Recruiter,0 - 1 yrs,Computer science| Fabrication| Quality check|...,R&D,Gurgaon,"Engineering Design , R&D","Recruitment, Staffing",R&D Executive
3,7b921f51b5c2fb862b4a5f7a54c37f75,2019-08-05 15:31:56 +0000,Technical Support,"2,00,000 - 4,00,000 PA.",0 - 5 yrs,Technical Support,Admin/Maintenance/Security/Datawarehousing,Mumbai,"IT Software - Application Programming , Mainte...","IT-Software, Software Services",Technical Support Engineer
4,2d8b7d44e138a54d5dc841163138de50,2019-07-05 02:48:29 +0000,Software Test Engineer -hyderabad,Not Disclosed by Recruiter,2 - 5 yrs,manual testing| test engineering| test cases|...,Programming & Design,Hyderabad,IT Software - QA & Testing,"IT-Software, Software Services",Testing Engineer


In [7]:
job_posting_df.drop(['Uniq Id', 'Crawl Timestamp'], axis= 1, inplace = True)

In [8]:
resume_df = pd.read_csv('/content/drive/MyDrive/hackathon_dataset/resume.csv')
resume_df.head(5)

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [9]:
import re
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText
    
resume_df['cleaned_resume'] = resume_df.Resume.apply(lambda x: cleanResume(x))

In [10]:
import nltk
from nltk.corpus import stopwords
import string

In [11]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
resume_df['cleaned_resume'] = resume_df['cleaned_resume'].apply(lambda text: remove_punctuation(text))

In [12]:
import nltk
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

resume_df['cleaned_resume'] = resume_df['cleaned_resume'].apply(lambda text: remove_stopwords(text))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
resume_df.drop(['Resume'], inplace= True, axis = 1)

In [14]:
resume_df.head(5)

,Category,cleaned_resume
0,Data Science,Skills Programming Languages Python pandas num...
1,Data Science,Education Details May 2013 May 2017 B E UIT RG...
2,Data Science,Areas Interest Deep Learning Control System De...
3,Data Science,Skills R Python SAP HANA Tableau SAP HANA SQL ...
4,Data Science,Education Details MCA YMCAUST Faridabad Haryan...


In [15]:
resume_df.dtypes

Category          object
cleaned_resume    object
dtype: object

In [74]:
# resume_df['Category'] = resume_df['Category'].fillna('').astype(str)
resume_df['cleaned_resume'] = resume_df['cleaned_resume'].fillna('').astype(str)

In [75]:

resume_df['cleaned_resume'].apply(type).unique()

array([<class 'str'>], dtype=object)

In [71]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [77]:
from nltk.tokenize import word_tokenize
# words = word_tokenize(resume_df['cleaned_resume'])
# len(words)

words = []
for resume in resume_df['cleaned_resume']:
    words.extend(word_tokenize(resume))
len(words)


326070

In [16]:
job_posting_df.head(5)

,Job Title,Job Salary,Job Experience Required,Key Skills,Role Category,Location,Functional Area,Industry,Role
0,Digital Media Planner,Not Disclosed by Recruiter,5 - 10 yrs,Media Planning| Digital Media,Advertising,Mumbai,"Marketing , Advertising , MR , PR , Media Plan...","Advertising, PR, MR, Event Management",Media Planning Executive/Manager
1,Online Bidding Executive,Not Disclosed by Recruiter,2 - 5 yrs,pre sales| closing| software knowledge| clien...,Retail Sales,"Pune,Pune","Sales , Retail , Business Development","IT-Software, Software Services",Sales Executive/Officer
2,Trainee Research/ Research Executive- Hi- Tec...,Not Disclosed by Recruiter,0 - 1 yrs,Computer science| Fabrication| Quality check|...,R&D,Gurgaon,"Engineering Design , R&D","Recruitment, Staffing",R&D Executive
3,Technical Support,"2,00,000 - 4,00,000 PA.",0 - 5 yrs,Technical Support,Admin/Maintenance/Security/Datawarehousing,Mumbai,"IT Software - Application Programming , Mainte...","IT-Software, Software Services",Technical Support Engineer
4,Software Test Engineer -hyderabad,Not Disclosed by Recruiter,2 - 5 yrs,manual testing| test engineering| test cases|...,Programming & Design,Hyderabad,IT Software - QA & Testing,"IT-Software, Software Services",Testing Engineer


In [17]:
job_posting_df['job_description'] = job_posting_df['Job Title'] + " " + job_posting_df['Role'] + " " + job_posting_df['Key Skills'] + " " + job_posting_df['Location']

In [57]:
job_posting_df['Role'] = job_posting_df['Role'].fillna('').astype(str)
job_posting_df['Industry'] = job_posting_df['Industry'].fillna('').astype(str)
job_posting_df['Functional Area'] = job_posting_df['Functional Area'].fillna('').astype(str)
job_posting_df['Location'] = job_posting_df['Location'].fillna('').astype(str)
job_posting_df['Role Category'] = job_posting_df['Role Category'].fillna('').astype(str)
job_posting_df['Key Skills'] = job_posting_df['Key Skills'].fillna('').astype(str)
job_posting_df['Job Experience Required'] = job_posting_df['Job Experience Required'].fillna('').astype(str)
job_posting_df['Job Salary'] = job_posting_df['Job Salary'].fillna('').astype(str)
job_posting_df['Job Title'] = job_posting_df['Job Title'].fillna('').astype(str)

In [58]:
# job_posting_df['job_description'] = job_posting_df['job_description'].astype(str)

job_posting_df['Job Title'].apply(type).unique()


array([<class 'str'>], dtype=object)

In [59]:
job_posting_df.dtypes

Job Title                  object
Job Salary                 object
Job Experience Required    object
Key Skills                 object
Role Category              object
Location                   object
Functional Area            object
Industry                   object
Role                       object
job_description            object
dtype: object

In [60]:
job_posting_df['job_description'].head()

0     Digital Media Planner Media Planning Executiv...
1     Online Bidding Executive Sales Executive Offi...
2     Trainee Research Research Executive Hi Tech O...
3     Technical Support Technical Support Engineer ...
4     Software Test Engineer hyderabad Testing Engi...
Name: job_description, dtype: object

In [61]:
import re
def cleanResume(resumeText):
    # resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText
# job_posting_df['job_description'] = job_posting_df['job_description'].apply(lambda x: cleanResume(x) if isinstance(x, str) else x)
obj_cols = job_posting_df.select_dtypes(include='object').columns
job_posting_df[obj_cols] = job_posting_df[obj_cols].applymap(cleanResume)
   
job_posting_df['job_description'] = job_posting_df['job_description'].apply(lambda x: cleanResume(x))

In [80]:
words = []
for job_posting in job_posting_df['job_description']:
    words.extend(word_tokenize(job_posting))
len(words)

614047

In [62]:
import nltk
from nltk.corpus import stopwords
import string

In [84]:
import pandas as pd
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

# Preprocess the resume dataset
resume_df['clean_text'] = resume_df['cleaned_resume'].apply(lambda x: ' '.join([word.lower() for word in word_tokenize(x)]))

# Preprocess the job posting dataset
job_posting_df['clean_text'] = job_posting_df['job_description'].apply(lambda x: ' '.join([word.lower() for word in word_tokenize(x)]))

# Combine the two datasets into one corpus
corpus = resume_df['clean_text'].tolist() + job_posting_df['clean_text'].tolist()

# Create TaggedDocument objects for each document in the corpus
documents = [TaggedDocument(words=word_tokenize(doc), tags=[str(i)]) for i, doc in enumerate(corpus)]

# Train the doc2vec model
model = Doc2Vec(documents, vector_size=100, window=10, min_count=5, epochs=50)

# Save the trained model
model.save('resume_job.model')

In [86]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 7.6 MB/s eta 0:00:00


In [87]:
import PyPDF2

In [88]:
resume_path = '/content/drive/MyDrive/hackathon_dataset/resume1.pdf'
resume1 = ''
pdfReader1 = PyPDF2.PdfReader(resume_path)
for i in range(len(pdfReader1.pages) ):
  pageObj = pdfReader1.pages[i]
  resume1 += pageObj.extract_text()

resume1 = resume1.lower()
resume1 = re.sub('[^a-z]', ' ', resume1)

In [89]:
resume_path = '/content/drive/MyDrive/hackathon_dataset/resume2.pdf'
resume2 = ''
pdfReader2 = PyPDF2.PdfReader(resume_path)
for i in range(len(pdfReader2.pages) ):
  pageObj = pdfReader2.pages[i]
  resume2 += pageObj.extract_text()

resume2 = resume2.lower()
resume2 = re.sub('[^a-z]', ' ', resume2)

In [90]:
resume_path = '/content/drive/MyDrive/hackathon_dataset/resume3.pdf'
resume3 = ''
pdfReader3 = PyPDF2.PdfReader(resume_path)
for i in range(len(pdfReader3.pages) ):
  pageObj = pdfReader3.pages[i]
  resume3 += pageObj.extract_text()

resume3 = resume3.lower()
resume3 = re.sub('[^a-z]', ' ', resume3)

In [91]:
resume_path = '/content/drive/MyDrive/hackathon_dataset/resume4.pdf'
resume4 = ''
pdfReader4 = PyPDF2.PdfReader(resume_path)
for i in range(len(pdfReader4.pages) ):
  pageObj = pdfReader4.pages[i]
  resume4 += pageObj.extract_text()

resume4 = resume4.lower()
resume4 = re.sub('[^a-z]', ' ', resume4)

In [92]:
resume_list = []
resume_list.append(resume1)
resume_list.append(resume2)
resume_list.append(resume3)
resume_list.append(resume4)

print(len(resume_list))

4


In [93]:
def extract_data(url):
  list1 = []
  count = 0
  resp = requests.get(url)
  if resp.status_code == 200:
    soup = BeautifulSoup(resp.text,'html.parser')
    l = soup.find(class_ = 'av-company-description-page mb-2')
    web = ''.join([i.text for i in l.find_all(['p', 'li'])])
    list1.append(web)
    return web
  else:
    print("Error")
jd_links = ['https://datahack.analyticsvidhya.com/jobathon/clix-capital/senior-manager-growthrisk-analytics-2',
'https://datahack.analyticsvidhya.com/jobathon/clix-capital/manager-growth-analytics-2',
'https://datahack.analyticsvidhya.com/jobathon/clix-capital/manager-risk-analytics-2',
'https://datahack.analyticsvidhya.com/jobathon/cropin/data-scientist-85']

jd_df = pd.DataFrame(columns = ['links', 'data'])
jd_df['links'] = jd_links

for i in range(len(jd_df)):
  jd_df['data'][i] = extract_data(jd_df['links'][i])

In [94]:
#Converting the text into lower case
jd_df.loc[:,"data"] = jd_df.data.apply(lambda x : str.lower(x))

#Removing the punctuations from the text
jd_df.loc[:,"data"] = jd_df.data.apply(lambda x : " ".join(re.findall('[\w]+',x))
)
#Removing the numerics present in the text
jd_df.loc[:,"data"] = jd_df.data.apply(lambda x : re.sub(r'\d+','',x))

In [95]:
from scipy.stats import norm
from scipy.stats import rv_continuous 
from scipy.special import gamma 
from scipy.stats import betaprime

In [104]:
model = Doc2Vec.load('resume_job.model')
cos_sim_list = []
v2 = model.infer_vector(jd_df['data'][0].split(' '), steps=20, alpha=0.025)
v2 = np.array(v2)
for i in range(len(resume_list)) :
  v1 = model.infer_vector(resume_list[i].split(' '), steps=20, alpha=0.025)
  v1 = np.array(v1)
  cosine_similarity = (np.dot(v1, v2)) / (np.linalg.norm(v1) * np.linalg.norm(v2))
  final_cos = round(cosine_similarity, 3)
  cos_sim_list.append(final_cos)

print(cos_sim_list)


[0.362, 0.411, 0.365, 0.294]
